# Install

In [1]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()

In [2]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.0 MB/s eta 0:00:00


# Import

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer

# Load Quantized Model

In [27]:
model_id = "CleverShovel/vicuna-7b-v1.3-sharded-bf16" # sharded model by vilsonrodrigues
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [28]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [29]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [30]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 8388608 || all params: 3508801536 || trainable%: 0.23907331075678143


# Prepare Data

In [31]:
from datasets import load_dataset

data = load_dataset("zadhart/ESG_Chat")

  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
tokenizer.pad_token = tokenizer.eos_token

# prompt_template = "### Instruction: {prompt}\n### Response:"

train_dataset = data['train'].map(lambda x: {"input_text": x['Perguntas']  + "\n" + x['Respostas']})

# Tokenize the datasets
train_encodings = tokenizer(train_dataset['input_text'], truncation=True, padding=True, max_length=256, return_tensors='pt')

In [21]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [22]:
# Convert the encodings to PyTorch datasets
train_dataset = TextDataset(train_encodings)

# Example Before Fine Tuning

In [23]:
def generate(index):

  example_text = data['train'][index]['Perguntas']
  correct_answer = data['train'][index]['Respostas']

  print("Question:")
  print(example_text)

  encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
  output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=150, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)
  predict = tokenizer.decode(output[0], skip_special_tokens=True)
  predict = predict[len(example_text)+1:]
  print("Answer:")
  print(predict)

  print("Best Answer:")
  print(correct_answer)

  print()

In [32]:
generate(10)

Question:
Define Corporate Social Responsibility in ESG context


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Answer:
Corporate Social Responsibility (CSR) refers to a company' The 2018-2023 World Outlook for Automotive Batteries
This study covers the world outlook for automotive batteries across more than 190 countries. For each year reported, estimates are given for the latent demand, or potential industry earnings (P.I.E.), for the country in question (in millions of U.S. dollars), the percent share the country is of the region, and of the globe.
This report provides you with valuable data and insights about the potential, opportunities, and challenges of the automotive batteries market across
Best Answer:
Corporate Social Responsibility (CSR) in the ESG (Environmental Social and Governance) context is a framework that promotes responsible business practices that promote sustainable economic growth while taking into account broader social and environmental impacts. CSR in the ESG context requires companies to consider their impact on the environment social issues and governance practices.



In [33]:
import random
random.seed(8)
rand_q = random.sample(list(range(len(data['train']))),10)

In [34]:
def get_response_set(model):
   predictions = []

   for index in rand_q:
      example_text = data['train'][index]['Perguntas']
      encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
      output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=150, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)
      predict = tokenizer.decode(output[0], skip_special_tokens=True)
      predict = predict[len(example_text)+1:]
      predictions.append(predict)

   return predictions


In [36]:
predictions = get_response_set(model)

In [37]:
predictions

['Identify key ESG issues relevant to green pricing programs\nDevelop a green pricing program that aligns with ESG principles\nImplement and monitor green pricing program\nCommunicate the benefits of green pricing program to stakeholders\nContinuously improve the green pricing program\nThe Green Pricing Program course is designed to provide an understanding of the concept of green pricing programs and how they align with Environmental, Social, and Governance (ESG) principles. The course will cover the key ESG issues relevant to green pricing programs, the development of a green pricing program that aligns with ESG principles, and the implementation and monitoring of the program. The course will also cover the benefits',
 'Green financing refers to the financing of projects, companies, or assets that have a positive environmental impact. It involves the allocation of capital to support sustainable development and address environmental challenges such as climate change, pollution, and re

In [38]:
reference = data["train"]["Respostas"]

In [39]:
references = []
questions = []
for i in rand_q:
   references.append(reference[i])
   questions.append(data['train'][i]['Perguntas'])

references

['Green pricing programs are programs offered by electricity providers that allow customers to pay a premium for electricity that is generated from renewable energy sources such as wind or solar power. These programs are often offered as a way to address environmental social and governance (ESG) concerns by reducing carbon emissions and promoting sustainable energy production. The premiums paid by customers are often used to purchase renewable energy certificates (RECs) or fund additional renewable energy projects. By participating in green pricing programs customers can demonstrate their commitment to sustainability and support the development of renewable energy resources which in turn can help to mitigate climate change and promote a more sustainable future.',
 'Green financing is a type of financing that is focused on environmentally sustainable projects and initiatives. It is guided by environmental social and governance (ESG) considerations and aims to support projects which have

In [40]:
import pandas as pd

def compare_df(question,pred,ref) -> pd.DataFrame:
   info = {
      "quetsion" : question,
      "response" : pred,
      "answer" : ref,
   }

   return pd.DataFrame(info)


df = compare_df(questions,predictions,references)

In [41]:
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=4dfcb0aca65e72e9359b61ee24420661cec7cd8c12f4393508512d2208f5f730
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [42]:
df.head()

,quetsion,response,answer
0,Define green pricing programs in ESG context,Identify key ESG issues relevant to green pric...,Green pricing programs are programs offered by...
1,Define green financing in ESG context,Green financing refers to the financing of pro...,Green financing is a type of financing that is...
2,Define data security in ESG context,Data security refers to the protection of sens...,In the context of environmental social and gov...
3,Define Living Wage in ESG context,The living wage is the minimum amount of incom...,Living wage is a concept in ESG (environmental...
4,Define biofuels in ESG context,"Biofuels are renewable, low-carbon energy sour...",Biofuels are renewable energy sources derived ...


In [43]:
import evaluate

In [44]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions,references=references,use_stemmer=True)
results

{'rouge1': 0.4521135134161909,
 'rouge2': 0.16549681545190473,
 'rougeL': 0.27429576028322344,
 'rougeLsum': 0.3053308783332366}

# Training

In [45]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=1,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

<ipython-input-21-424aeced090c>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.470800
2,1.439100
3,1.449400
4,1.442000
5,1.372300
6,1.351000
7,1.273700
8,1.227400
9,1.167900
10,1.178200


TrainOutput(global_step=100, training_loss=1.0877360409498216, metrics={'train_runtime': 4509.1739, 'train_samples_per_second': 1.419, 'train_steps_per_second': 0.022, 'total_flos': 3.166353923820749e+16, 'train_loss': 1.0877360409498216, 'epoch': 33.33})

# Example After Fine Tuning

In [46]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              

In [47]:
generate(39)

Question:
Define emissions in ESG context


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Answer:
Emissions in the ESG (Environmental Social and Governance) context refer to the release of harmful gases and pollutants into the atmosphere primarily from human activities such as burning fossil fuels deforestation and industrial processes. These emissions contribute to climate change which has significant social and environmental impacts including rising temperatures extreme weather events and loss of biodiversity. Therefore managing and reducing emissions is a critical aspect of ESG investing and management.

Emissions are typically measured in terms of their intensity or intensity per unit of production or consumption. For example a company's carbon dioxide (CO2) emissions per unit of production or consumption is a key metric for assessing its
Best Answer:
Emissions in the ESG (Environmental Social and Governance) context refer to the release of harmful gases or pollutants into the environment which contribute to climate change and related environmental impacts. Emissions ca

In [48]:
prediction2 = get_response_set(model)

In [49]:
prediction2

['Green pricing programs are a type of environmental social and governance (ESG) initiative that allows customers to pay a premium for electricity that is generated from renewable sources such as wind solar or hydroelectric power. The premium is typically passed on to the utility company which then invests in renewable energy projects to reduce the carbon footprint of the power grid.\n\nGreen pricing programs are designed to encourage customers to adopt sustainable energy practices and reduce their environmental impact. By choosing to participate in a green pricing program customers can support the transition to a low-carbon energy system reduce their carbon footprint and contribute to a more sustainable future.\n\nIn addition to supporting renew',
 'Green financing refers to the investment of financial resources in projects or companies that promote environmental social and governance (ESG) principles. In the context of ESG green financing refers to the investment of capital in compan

In [50]:
results2 = rouge.compute(predictions=prediction2,references=references,use_stemmer=True)

In [51]:
results2

{'rouge1': 0.5697584496189545,
 'rouge2': 0.2639641892305975,
 'rougeL': 0.3603096245769519,
 'rougeLsum': 0.39111540660527333}

In [52]:
bleu = evaluate.load('bleu')
results_bleu = bleu.compute(predictions=predictions, references=references)
results_bleu2 = bleu.compute(predictions=prediction2, references=references)

print(results_bleu)
print(results_bleu2)

{'bleu': 0.07690578149807137, 'precisions': [0.4226713532513181, 0.13386524822695037, 0.0518783542039356, 0.01895306859205776], 'brevity_penalty': 0.8904814832507468, 'length_ratio': 0.8960629921259843, 'translation_length': 1138, 'reference_length': 1270}
{'bleu': 0.20775165644711646, 'precisions': [0.565587734241908, 0.26288659793814434, 0.16031195840554593, 0.10839160839160839], 'brevity_penalty': 0.9214822901003235, 'length_ratio': 0.9244094488188976, 'translation_length': 1174, 'reference_length': 1270}


In [53]:
toxicity = evaluate.load("toxicity")
results_tox = toxicity.compute(predictions=predictions, aggregation="ratio")
results_tox2 = toxicity.compute(predictions=prediction2, aggregation="ratio")
print(results_tox)
print(results_tox2)

{'toxicity_ratio': 0.0}
{'toxicity_ratio': 0.0}


In [54]:
df.insert(loc = 2, column="vicuna7b_model_response", value=prediction2)

In [ ]:
df.insert(loc = 3, column="vicuna7b_response", value=predictions3)

In [55]:
df

,quetsion,response,vicuna7b_model_response,answer
0,Define green pricing programs in ESG context,Identify key ESG issues relevant to green pric...,Green pricing programs are a type of environme...,Green pricing programs are programs offered by...
1,Define green financing in ESG context,Green financing refers to the financing of pro...,Green financing refers to the investment of fi...,Green financing is a type of financing that is...
2,Define data security in ESG context,Data security refers to the protection of sens...,In the context of Environmental Social and Gov...,In the context of environmental social and gov...
3,Define Living Wage in ESG context,The living wage is the minimum amount of incom...,In the context of ESG (Environmental Social an...,Living wage is a concept in ESG (environmental...
4,Define biofuels in ESG context,"Biofuels are renewable, low-carbon energy sour...",Biofuels are renewable energy sources that are...,Biofuels are renewable energy sources derived ...
5,Define Refinitiv Scoring in ESG context,\n* Refinitiv Scoring is a proprietary ESG sco...,Refinitiv Scoring is a methodology used to eva...,Refinitiv Scoring in ESG context refers to an ...
6,Define corporate governance in ESG context,Corporate governance refers to the system of r...,Corporate governance in the ESG (Environmental...,Corporate governance in ESG (Environmental Soc...
7,Define governance through exit in ESG context,"In the context of Environmental, Social and Go...",Governance through exit in the ESG (Environmen...,Governance through exit in ESG context refers ...
8,Define stakeholders in ESG context,Explain the role of stakeholders in ESG\nDiscu...,Stakeholders in the ESG (Environmental Social ...,Stakeholders in an ESG (Environmental Social a...
9,Define floods in ESG context,Floods are a natural hazard that can cause sig...,Floods in the ESG (Environmental Social and Go...,In the context of ESG (Environmental Social an...


In [ ]:
c1 = "rouge1"
c2 = "rouge2"
cL = "rougeL"
cLsum = "rougeLsum"
df_score = pd.DataFrame({
    c1 : [results[c1], results2[c1], results3[c1]],
    c2 : [results[c2], results2[c2], results3[c2]],
    cL : [results[cL], results2[cL], results3[cL]],
    cLsum : [results[cLsum], results2[cLsum], results3[cLsum]],
})
df_score.index = ["falcon7b" , "ESG_model", "vicuna7b"]
df_score

,rouge1,rouge2,rougeL,rougeLsum
falcon7b,0.466976,0.145024,0.273879,0.294289
ESG_model,0.525273,0.203351,0.325743,0.354960
vicuna7b,0.457632,0.168178,0.277941,0.301291


# Merge Adapters and Save Model to Hub

In [56]:
from huggingface_hub import notebook_login

notebook_login()

In [57]:
model.save_pretrained("vicuna-7b-instruct-ft-adapters-ESG-chatting", push_to_hub=True) # save LORA adapters
model.push_to_hub("vicuna-7b-instruct-ft-adapters-ESG-chatting")

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/FelixChao/vicuna-7b-instruct-ft-adapters-ESG-chatting/commit/39181a93826799ba097346dcd71d569fa0844a16', commit_message='Upload model', commit_description='', oid='39181a93826799ba097346dcd71d569fa0844a16', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from peft import PeftModel, PeftConfig, LoraConfig
from transformers import AutoModelForCausalLM

config = LoraConfig.from_pretrained("FelixChao/falcon-7b-instruct-ft-adapters-ESG-chatting")
# reload the base model (you might need pro subscription for this)

In [ ]:
from peft import PeftModel

# load perf model with new adapters
model = PeftModel.from_pretrained(
    model,
    "FelixChao/falcon-7b-instruct-ft-adapters-ESG-chatting",
)

In [ ]:
model = model.merge_and_unload() # merge adapters with

ValueError: ignored

In [ ]:
model.push_to_hub("falcon-7b-instruct-ft")

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vrsen/falcon-7b-instruct-ft/commit/93f6d84447abd80fff5ff062c81dc86b0ac72e82', commit_message='Upload RWForCausalLM', commit_description='', oid='93f6d84447abd80fff5ff062c81dc86b0ac72e82', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct", trust_remote_code=True)
tokenizer.push_to_hub("falcon-7b-instruct-ft", use_auth_token=True)

CommitInfo(commit_url='https://huggingface.co/vrsen/falcon-7b-instruct-ft/commit/ae7eca5a54426134be2babe2239380487dad3306', commit_message='Upload tokenizer', commit_description='', oid='ae7eca5a54426134be2babe2239380487dad3306', pr_url=None, pr_revision=None, pr_num=None)